In [17]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *
from vis3d_utilities import *
from metadata import *
from data_manager import *

import time

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open('colors.txt', 'r') as f:
    colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [3]:
train_sample_scheme = 1
global_transform_scheme = 1
local_transform_scheme = 2

In [4]:
atlas_name = 'atlasV2'

In [5]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

structure_colors = {n: np.random.randint(0, 255, (3,)) for n in structures}
# with open('colors.txt', 'r') as f:
#     colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [6]:
structures_sided = sum([[n] if n in singular_structures else [convert_to_left_name(n), convert_to_right_name(n)]
                        for n in structures], [])

In [24]:
stack_colors = {n: np.array(high_contrast_colors[i%len(high_contrast_colors)]) for i, n in enumerate(all_stacks)}

# Genrate mesh for subject volume mapped back to atlas space

In [34]:
# Render globally transformed atlas structures

# stack = 'MD590'
for stack in all_stacks:
    
    for name_s in structures_sided:
    # for name_s in ['12N']:

        print name_s

        reversed_volume = \
        DataManager.load_transformed_volume(stack_m=atlas_name, type_m='score', 
                                            stack_f=stack, type_f='score',
                                            downscale=32, 
                                            train_sample_scheme_f=train_sample_scheme,
                                            global_transform_scheme=global_transform_scheme,
                                            local_transform_scheme=local_transform_scheme,
                                            label=name_s, transitive='by')

    #     xmin, xmax, ymin, ymax, zmin, zmax = bbox_3d(reversed_volume)
    #     vol = reversed_volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1]

    #     dense_vol = fill_sparse_score_volume(vol)

        t = time.time()

        polydata = volume_to_polydata(reversed_volume, origin=(0,0,0), num_simplify_iter=3, smooth=True, level=.5)

        sys.stderr.write('time: %.2f\n' % (time.time() - t))

        mesh_fn = DataManager.get_transformed_volume_mesh_filepath(stack_m=atlas_name, type_m='score', 
                                                             stack_f=stack, type_f='score', 
                                                             downscale=32,
                                                            train_sample_scheme_f=train_sample_scheme,
                                                            global_transform_scheme=global_transform_scheme,
                                                            local_transform_scheme=local_transform_scheme,
                                                            label=name_s, transitive='by')     

        save_mesh(polydata, mesh_fn)

5N_L


marching cube: 5.82 seconds
mesh_to_polydata: 0.01 seconds
simplify 0 @ 7311: 0.06 seconds
simplify 1 @ 1464: 0.05 seconds
simplify 2 @ 294: 0.01 seconds
time: 6.48


5N_R


marching cube: 8.19 seconds
mesh_to_polydata: 0.01 seconds
simplify 0 @ 7424: 0.06 seconds
simplify 1 @ 1486: 0.06 seconds
simplify 2 @ 299: 0.01 seconds
time: 8.90


6N_L


...

marching cube: 7.49 seconds
mesh_to_polydata: 0.03 seconds
simplify 0 @ 26344: 0.23 seconds
simplify 1 @ 5265: 0.23 seconds
simplify 2 @ 1052: 0.04 seconds
time: 9.13


SC


marching cube: 7.78 seconds
mesh_to_polydata: 0.33 seconds
simplify 0 @ 120918: 2.95 seconds
simplify 1 @ 24185: 2.20 seconds
simplify 2 @ 4837: 0.33 seconds
time: 14.18


IC


marching cube: 8.50 seconds
mesh_to_polydata: 0.12 seconds
simplify 0 @ 73684: 1.08 seconds
simplify 1 @ 14742: 0.82 seconds
simplify 2 @ 2948: 0.17 seconds
time: 11.45


# Load

In [73]:
# Load locally transformed atlas structure meshes

reversed_polydata_list = {}
for stack in all_stacks:
    reversed_polydata_list[stack] = DataManager.load_transformed_volume_meshes(stack_m=atlas_name, type_m='score', 
                                                         stack_f=stack, type_f='score', 
                                                         downscale=32,
                                                        train_sample_scheme_f=train_sample_scheme,
                                                        global_transform_scheme=global_transform_scheme,
                                                        local_transform_scheme=local_transform_scheme,
                                                        labels=structures_sided, transitive='by',
                                                      return_polydata_only=True)

In [74]:
ann_vol_xmin, ann_vol_xmax, ann_vol_ymin, ann_vol_ymax, ann_vol_zmin, ann_vol_zmax = \
DataManager.load_volume_bbox(stack='MD589', type='annotation', downscale=32)

print 'annotation volume bbox :', ann_vol_xmin, ann_vol_xmax, ann_vol_ymin, ann_vol_ymax, ann_vol_zmin, ann_vol_zmax

annotation volume bbox : 622 1042 155 502 127 503


In [75]:
shell_polydata = DataManager.load_shell_mesh(stack='MD589', downscale=32)

In [76]:
# Load atlas meshes

In [77]:
# Load canonical info
canonical_locations = read_dict_from_txt(os.path.join(MESH_ROOTDIR, atlas_name, 'canonical_locations.txt'))
canonical_centroid = np.loadtxt(os.path.join(MESH_ROOTDIR, atlas_name, 'canonical_centroid.txt'))
canonical_normal = np.loadtxt(os.path.join(MESH_ROOTDIR, atlas_name, 'canonical_normal.txt'))

In [78]:
# Load atlas meshes

atlas_space_structure_meshes = DataManager.load_meshes(stack=atlas_name, labels=structures_sided)

In [79]:
# Load atlas structure surround meshes

atlas_space_structure_surround_meshes = DataManager.load_meshes(stack=atlas_name, \
                                        labels=[n + '_surround' for n in structures_sided])

In [93]:
# Create actors

all_atlas_structure_mesh_actors = {n: actor_mesh(polydata, (1.,1,1),
                                             wireframe=False,
                                                 opacity=.3,
                                              origin=canonical_centroid) 
                                   for n, polydata in atlas_space_structure_meshes.iteritems()}

all_atlas_structure_surround_mesh_actors = [actor_mesh(polydata, color=(0,1,0),
                                                      wireframe=True,
                                                      origin=canonical_centroid) 
                                            for n, polydata in atlas_space_structure_surround_meshes.iteritems()]
    
shell_actor = actor_mesh(shell_polydata, (1,1,1), opacity=.15, wireframe=False)

In [94]:
all_reversed_structure_actors = {st: [actor_mesh(p, 
                       wireframe=False, 
#                        color=structure_colors[convert_name_to_unsided(n)]/255., 
#                         color=(0,1,0), 
                        color=stack_colors[st]/255.,
                       opacity=.3,
                      origin=(ann_vol_xmin, 
                              ann_vol_ymin, 
                              ann_vol_zmin))
                                      for n, p in ps.iteritems() if n not in outliers or st not in outliers[n]]
            for st, ps in reversed_polydata_list.iteritems() }

In [82]:
launch_vtk(all_atlas_structure_mesh_actors.values() + \
           [shell_actor] + \
           all_reversed_structure_actors)

TypeError: can only concatenate list (not "dict") to list

In [83]:
outliers = read_dict_from_txt('/home/yuncong/Brain/outliers.txt', converter=str)

In [44]:
# names_of_interest = ['7N_L', '7N_R', '5N_L', '5N_R', '12N', 'Tz_L', 'Tz_R', '7n_L', '7n_R', 'LRt_L', 'LRt_R',
#                     'Pn_L', 'Pn_R', 'SNR_L', 'SNR_R']

names_of_interest = ['7n_L']

# names_of_interest = [n for n in structures_sided \
#                      if '_L' not in n and \
#                      convert_name_to_unsided(n) not in ['AP', 'DC', 'Sp5O', 'Sp5I', 'Sp5C', 'RtTg', 'VCP', 'VCA']]


particular_reversed_structure_actors_all = {}

for name_s_interest in names_of_interest:

    if name_s_interest in outliers:
        print 'Outlier stack for %s:' % name_s_interest, outliers[name_s_interest]

    particular_reversed_structure_actors = [actor_mesh(p, 
                           wireframe=True, 
    #                        color=structure_colors[convert_name_to_unsided(n)]/255., 
    #                         color=(0,1,0), 
                            color=stack_colors[st]/255.,
                           opacity=1.,
                          origin=(ann_vol_xmin, 
                                  ann_vol_ymin, 
                                  ann_vol_zmin))
                for st, ps in reversed_polydata_list.iteritems() for n, p in ps.iteritems() 
                                            if n == name_s_interest and \
                                            (name_s_interest not in outliers or st not in outliers[name_s_interest])]
    
    particular_reversed_structure_actors_all[name_s_interest] = particular_reversed_structure_actors

Outlier stack for 7n_L: ['MD591']


In [45]:
launch_vtk([all_atlas_structure_mesh_actors[n] for n in names_of_interest] + \
           [shell_actor] + \
           sum(particular_reversed_structure_actors_all.values(), []), init_angle='horizontal_topDown')

In [95]:
launch_vtk(all_atlas_structure_mesh_actors.values() + all_reversed_structure_actors['MD589'] + \
           [shell_actor])

In [119]:
launch_vtk(all_atlas_structure_mesh_actors.values() + sum(all_reversed_structure_actors.values(), []) + \
           [shell_actor])